## **VRP**

***Indices and Parameters***

- $I = J = K = \{1, 2, ..., n\}$: location of demands
- $d_{ij}$: Distance from demand point $i$ to demand $j$, 

***Decision Variables***

- $x_{i,j}$: Binary variable that equals 1 if arc $(i,j)$ is selected


***Objective Function***

Minimize the total traveled distance

$$ \min \sum_{i,j} d_{ij} x_{ij} $$

**Constraints**

1. $$  \sum_{i \in I ,\ i != k} x_{ik} = 1 \quad \forall k \in K  $$
2. 1. $$  \sum_{j \in J ,\ J != k} x_{kj} = 1 \quad \forall k \in K  $$

**code**

In [5]:
import pulp as p

class VRP:
    def __init__(self, I,J,K,d):
        self.I = I 
        self.J = J 
        self.K = K
        self.d = d

        #initialize 
        self.problem = p.LpProblem("VRP", p.LpMinimize)

        #DVs
        self.x = p.LpVariable.dicts("x", [(i,j) for i in self.I for j in self.J if i != j], cat= p.LpBinary)

    def model(self):
        #obj
        self.problem += p.lpSum(self.d[(i,j)] * self.x[(i,j)] for i in self.I for j in self.J if i != j)
        #constraints
        #1 and 2
        for k in self.K:
            self.problem += p.lpSum(self.x[i, k] for i in self.I if k != i) == 1
            self.problem += p.lpSum(self.x[k, j] for j in self.J if j != k) == 1
     # Vehicle k arrives at node j
        #3: subtour elimination
        #3_1:Subtour elimination: Connected Component Constraints
        for j in self.J:
            if j not in self.K:
                # Sum of incoming flows to j must equal the sum of outgoing flows from j
                self.problem += p.lpSum(self.x[(i, j)] for i in self.I if i != j) == p.lpSum(self.x[(j, i)] for i in self.I if i != j)

        #3_2:
    def solve(self):
        self.problem.solve()

        # Use CPLEX solver
        # self.problem.solve(p.CPLEX_PY(msg=True))
        
        print(f"Status: {p.LpStatus[self.problem.status]}")
        if self.problem.status == p.LpStatusOptimal:
            print(f"Optimal value is: {p.value(self.problem.objective)}")
            for v in self.x:
                if p.value(self.x[v]) != 0:
                    print(f'{v}: {p.value(self.x[v])}')
        

In [6]:
# Example usage
I = [0, 1, 2, 3]  # Node 0 could be the depot
J = [0, 1, 2, 3]
K = [0]  # Assuming one vehicle starts at node 0
d = {(i, j): 1 for i in I for j in J if i != j}  # Simple distance of 1 for demonstration

vrp = VRP(I, J, K, d)
vrp.model()
vrp.solve()


Status: Optimal
Optimal value is: 2.0
(0, 2): 1.0
(2, 0): 1.0
